In [ ]:
# For parsing and saving
import datetime as dt
import csv

from pprint import pprint

from util.helpers import startWebdriver
from util.log_videos import update_video_log

from util.custom_values import DATA_DIR, CHANNEL_ID, CHROMEDRIVER_PATH, USER_DATA_PATH, CHROME_PROFILE
from util.constants import METRICS, TimePeriod, TRAFFIC_SOURCES_IMP, \
    TRAFFIC_SOURCES, TRAFFIC_SOURCES_INV, Dimensions, ADV_URL, VIDEOS_URL, VIDEO_URL

In [ ]:
from util.api_scrape import scrape_channels_info

In [ ]:
driver = startWebdriver()
output = scrape_channels_info(driver, [CHANNEL_ID]*5)
output

TESTING API SCRAPING

In [ ]:
from util.api_scrape import switch_to_iframe, execute
from util.api_scrape_utils import clean_response, parse_video_item_str
import datetime as dt
from typing import List, Dict, Union

API_URL = "https://developers.google.com/youtube/v3/docs/{mode}/list?apix=true"

def scrape_videos_basics_by_api(driver, video_ids: List[str]) -> Dict[str, Dict[str,Union[str,dt.datetime]]]:
    """
    Return dictionary of video upload date and title by video id.
    """
    videos = {}

    try:
        driver.get(API_URL.format(mode="videos"))

        switch_to_iframe(driver)
        
        # Enter arguments
        driver.find_element(By.ID, "part[0]").send_keys("snippet") # Enter parts
        driver.find_element(By.CSS_SELECTOR, "label.mat-checkbox-layout").click() # Disable auth

        for vid_id in video_ids:
            # Enter playlist id
            element = driver.find_element(By.ID, f"id[0]")
            element.clear()
            element.send_keys(vid_id)

            # Get result in json string
            json_string = clean_response(execute(driver))

            # Parse the video json into our format
            video_dict = parse_video_item_str(json_string)
            if not video_dict:
                print(f"Couldn't find {vid_id}: might be private [{__file__}]")
                continue
            videos[vid_id] = video_dict
    except Exception as e:
        print("error whilst scraping video info from YouTube api: ", e, f"[{__file__}]")
    finally:
        # driver.quit()
        return videos

In [ ]:
driver = startWebdriver()
videos = scrape_videos_basics_by_api(driver, video_ids)

display(videos)